<a href="https://colab.research.google.com/github/USCbiostats/PM520/blob/main/Lab_11_Variational_Inference_PtIV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Lucky, or: Non-Conjugate Variational Inference Pt II